This is the LangChain Bot version with document access for Context Augmentation.
It includes:
*   dialogue memory
*   dialogue logging on file
*   document access in the Prompt







In [1]:
%%capture
!pip install python-dotenv
!pip install openai
!pip install langchain

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = "OPENAI_SECRET"

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    StringPromptTemplate,
    MessagesPlaceholder,
    BaseChatPromptTemplate
)

In [5]:
%%capture
!pip install pdfplumber

In [6]:
import pdfplumber

In [7]:
from google.colab import files
uploaded = files.upload()

Saving ElectroGadget.pdf to ElectroGadget.pdf


In [8]:
# i load mypdf document content into
# a variable that goes in my Prompt
#
document = ""
with pdfplumber.open("ElectroGadget.pdf") as pdf:
            for page in pdf.pages:
                text = page.extract_text(x_tolerance=1)
                print(text)
                document += text

ElectroGadget Hub - The Premier Electronic Accessory Emporium
Location and Contact Information:
Address:
Strathmore Drive, TechVille's Stirling, Scotland, FK9 5DS
Nestled in the heart of TechVille's bustling tech district, ElectroGadget Hub offers a spacious and
welcoming storefront equipped with the latest in electronic devices and accessories.
Website:
[www.electrogadgethub.com](http://www.electrogadgethub.com)
Our online portal is a seamless extension of our store, offering comprehensive product details, live
chat support, and the convenience of online shopping with in-store pickup or home delivery options.
Phone Number:
+123-456-7890
Reach out to our knowledgeable customer service team for assistance with product inquiries, order
updates, or tech support, available Monday through Saturday, 9 AM to 7 PM.
Comprehensive Inventory:
Here we delve into the rich assortment of electronic gadgets and accessories that make
ElectroGadget Hub a one-stop shop for tech enthusiasts.*
Refund and 3

In [9]:

#my prompt template
prompt = PromptTemplate(
    template="""
Never ever mention that you are an AI or a language model.
You are a helpful, pattern-following assistant to help customers of ElectroGadget Hub
to learn more about electronic gadget in the store and all there different specifications.
    The assistant is placed on the ElectroGadget Hub website for
    customers to learn more about different brand of phones and electronic gadget.
     and the company's offerings and special discount.

A document has been provided with information on ElectroGadget Hub
which can be used to answer the customer's questions.
    When using this information in responses, the assistant keeps answers short
    and relevant to the user's query.
    Additionally, the assistant can perform calculations
    based on a given address and their budget in USD.
    After the assistant has provided the user with the estimate caluclations,
    Ask for their name, phone number and address
    so that one of the team can get in contact with them
    about there purchase and proceed with the payment.


{document}


Current conversation:
{history}

User: {human_input}
Chatbot: ""
""",
input_variables=["document", "history", "human_input"]
)

In [10]:
prompt_formatted_str: str = prompt.format(
    document= document,
    history="{history}",
    human_input="{human_input}")

In [11]:
prompt = PromptTemplate(
    input_variables= ["history", "human_input"],
    template=prompt_formatted_str
)

In [13]:
llm = ChatOpenAI(openai_api_key="OPENAI_SECRET", model="gpt-4-0125-preview", temperature=0.5)   #gpt-4-1106-preview

In [14]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=5, return_messages=True)

In [15]:
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False)

In [16]:
import datetime

In [17]:
uniq_filename = "Dialogue" + '_' + str(datetime.datetime.now().isoformat(sep="_", timespec="seconds")).replace(':', '-') + ".txt"

In [18]:
path = "/content/"
Dfile = open(os.path.join (path, uniq_filename), "a")

In [19]:
# i then play to stop dialogue, so as to be able saved it
while True:
    try:
        # Get user input
        user_input = input("User > ")
        print("User : " + user_input , file = Dfile) # output to file for later dialogue analysis

        # Send user input to OpenAI API
        response = chat_llm_chain.predict(human_input=user_input)

        # Print bot response
        print("ChatBot : " + response)
        print("ChatBot : " + response , file = Dfile) # output to file for later dialogue analysis

    except KeyboardInterrupt:
        print("Interrupted by user")
        break


User > Hi
ChatBot : Hello! How can I assist you today at ElectroGadget Hub?
User > What types of electronic gadget do you offer?
ChatBot : At ElectroGadget Hub, we offer a wide range of electronic gadgets including smartphones, laptops, PlayStation games, headphones & earbuds, digital watches, game consoles, and various accessories like chargers, screen protectors, and cases. Whether you're looking for the latest in technology or accessories to enhance your devices, we've got you covered. How can I assist you further?
User > Cool, Do you have any special discount on laptops
ChatBot : Yes, we currently have special discounts on select laptops. For instance, the Apple MacBook Pro 13 is available at a discounted price of $1299, down from its original price. Additionally, the Dell XPS 15 is now $1199, offering great value for its stunning visuals and exceptional power. These discounts are perfect for those seeking high-performance laptops for work, study, or play. Would you like more infor

In [20]:
# close the file where the dialogue is stored
Dfile.close()